In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import dataiku
from dataiku import pandasutils as pdu
import pandas as pd
#import plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [3]:
mydataset = dataiku.Dataset("RHEOPROFILER_Data_PVYP")
mydataset_df = mydataset.get_dataframe()

treatment = dataiku.Dataset("TretmentTimeline")
treatment_df = treatment.get_dataframe()

list_component = list(treatment_df.columns.values)
list_component.remove('Timestamp')
list_component.remove('Well Name')
#list_component = [("Lime")]

In [4]:
treatment_df = treatment_df[(treatment_df["Well Name"] == 'UTMN-2287')]

treatment_df = treatment_df.replace(-999,np.nan).dropna(axis=1,how="all")
treatment_df = treatment_df.dropna(thresh=3)

treatment_min_y, treatment_max_y = (treatment_df[list_component].min()).min(),(treatment_df[list_component].max()).max() 

treatment_df = treatment_df.fillna(-999)


treatment_df = treatment_df.rename(columns={'Timestamp': 'Full Date_parsed'})

In [5]:
mydataset_df = mydataset_df[["Well Name","Date","Full Date_parsed","Sample Temp","YP"]]  

mydataset_df = mydataset_df[(mydataset_df["Well Name"] == 'UTMN-2287')&(mydataset_df["Sample Temp"] == 120)]


In [6]:
mydataset_df = mydataset_df[pd.to_numeric(mydataset_df['YP'], errors='coerce').notnull()]
mydataset_df['YP'] = pd.to_numeric(mydataset_df.YP)
min_y, max_y = mydataset_df["YP"].min(),mydataset_df["YP"].max() 

In [7]:
mydataset_df = mydataset_df.reset_index(drop=True)

In [8]:
mydataset_df['Trigger1'] = np.where(mydataset_df['YP'] < 14, mydataset_df['YP'],-999)
mydataset_df['Trigger2'] = np.where(mydataset_df['YP'] > 22, mydataset_df['YP'],-999)
#print(mydataset_df)
mydataset_df['Trigger3'] = -999
mydataset_df['Trigger4'] = -999
mydataset_df['Trigger5'] = -999
mydataset_df['Trigger6'] = -999
for index in range(2, len(mydataset_df.index)):
    trigger1 = mydataset_df.iloc[index]['Trigger1']
    trigger2 = mydataset_df.iloc[index]['Trigger2']
    
    value1=mydataset_df.iloc[index]['YP']
    value2 =mydataset_df.iloc[index-1]['YP']
    value3=mydataset_df.iloc[index-2]['YP']
    if trigger1!=-999 or trigger2 !=-999:
        
        continue 
        
    else:
        if value1> value2 and value2> value3:
            if trigger1==-999:
                mydataset_df.set_value(index, 'Trigger3', value1)    
        if value1< value2 and value2< value3:

            mydataset_df.set_value(index, 'Trigger4', value1)
            #mydataset_df.iloc[index]['Trigger3'] = value1

        if value1>value2*1.2 :
            mydataset_df.set_value(index, 'Trigger5', value1)

        if value1<value2*0.8 :
            mydataset_df.set_value(index, 'Trigger6', value1)


/home/dataiku/dss/code-envs/python/scr_d3/lib/python3.6/site-packages/ipykernel_launcher.py:29: FutureWarning:

set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead

/home/dataiku/dss/code-envs/python/scr_d3/lib/python3.6/site-packages/ipykernel_launcher.py:22: FutureWarning:

set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead

/home/dataiku/dss/code-envs/python/scr_d3/lib/python3.6/site-packages/ipykernel_launcher.py:25: FutureWarning:

set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead

/home/dataiku/dss/code-envs/python/scr_d3/lib/python3.6/site-packages/ipykernel_launcher.py:32: FutureWarning:

set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead



In [9]:
mydataset_df = pd.merge(mydataset_df,treatment_df,on=['Full Date_parsed',"Well Name"],how='outer')
mydataset_df = mydataset_df.fillna(-999)
#mydataset_df = mydataset_df.append(treatment_df)
mydataset_df = mydataset_df.sort_values(by=['Full Date_parsed'])
mydataset_df['theDate']= pd.to_datetime(mydataset_df['Full Date_parsed']).dt.strftime('%Y-%m-%d %H:%M')
#print(mydataset_df[['Full Date_parsed', 'theDate']])
min_x, max_x = mydataset_df["Full Date_parsed"].min(),mydataset_df["Full Date_parsed"].max()   
#min_date,max_date = mydataset_df["Full Date_parsed"].min(),mydataset_df["Full Date_parsed"].max()

In [10]:
fig_dict = {
    "layout": {},
    "frames": []
}

fig_dict["layout"]["xaxis"] = {"range": [min_x, max_x], "title": "Date"}
fig_dict["layout"]["yaxis"] = {"range": [min_y, max_y],"title": "RPYP"}
fig_dict["layout"]["hovermode"] = "closest"
fig_dict["layout"]["updatemenus"] = [
    {
        "buttons": [
            {
                "args": [None, {"frame": {"duration": 500, "redraw": False},
                                "fromcurrent": True, "mode": "immediate", "transition": {"duration": 0
                                                                    }}],
                "label": "Play",
                "method": "animate"
            },
            {
                "args": [[None], {"frame": {"duration": 0, "redraw": False},
                                  "mode": "immediate",
                                  "transition": {"duration": 0}}],
                "label": "Pause",
                "method": "animate"
            }
        ],
        "direction": "left",
        "pad": {"r": 10, "t": 87},
        "showactive": False,
        "type": "buttons",
        "x": 0.1,
        "xanchor": "right",
        "y": 0,
        "yanchor": "top"
    }
]



sliders_dict = {
    "active": 0,
    "yanchor": "top",
    "xanchor": "left",
    "currentvalue": {
        "font": {"size": 20},
        "prefix": "Date:",
        "visible": True,
        "xanchor": "right"
    },
    "transition": {"duration": 0, "easing": "cubic-in-out"},
    "pad": {"b": 10, "t": 50},
    "len": 0.9,
    "x": 0.1,
    "y": 0,
    "steps": []
}


# make data
sub_fig_data= {
    "data":[]
}
data_dict = {
        "x": list(mydataset_df['Full Date_parsed'][:2]),
        "y": list(mydataset_df['YP'][:2]),
        "mode": "markers",
        "marker": {
            #"sizemode": "area",
            #"sizeref": 200000,
            "size": 4
        }
        }
sub_fig_data["data"].append(data_dict)
data_dict = {
        "x": list(mydataset_df['Full Date_parsed'][:2]),
        "y": list(mydataset_df['Trigger1'][:2]),
        "text":"YP is too low, please add TRUVIS",
        "mode": "markers",
        "marker": {
            #"sizemode": "area",
            #"sizeref": 200000,
            "size": 10,
            "opacity":0.5
        }
        }
sub_fig_data["data"].append(data_dict)
data_dict = {
        "x": list(mydataset_df['Full Date_parsed'][:2]),
        "y": list(mydataset_df['Trigger2'][:2]),
        "text":"YP is too high, please add Diesel",
        "mode": "markers",
        "marker": {
            #"sizemode": "area",
            #"sizeref": 200000,
            "size": 10,
            "opacity":0.5
        }
        }
sub_fig_data["data"].append(data_dict)
data_dict = {
        "x": list(mydataset_df['Full Date_parsed'][:2]),
        "y": list(mydataset_df['Trigger3'][:2]),
        "text":"Increasing YP trend – Within limits but consider adding Diesel",
        "mode": "markers",
        "marker": {
            #"sizemode": "area",
            #"sizeref": 200000,
            "size": 10,
            "opacity":0.5
        }
        }
sub_fig_data["data"].append(data_dict)

data_dict = {
        "x": list(mydataset_df['Full Date_parsed'][:2]),
        "y": list(mydataset_df['Trigger4'][:2]),
        "text":"Decreasing YP trend – Within limits but consider adding TRUVIS",
        "mode": "markers",
        "marker": {
            #"sizemode": "area",
            #"sizeref": 200000,
            "size": 10,
            "opacity":0.5
        }
        }
sub_fig_data["data"].append(data_dict)

data_dict = {
        "x": list(mydataset_df['Full Date_parsed'][:2]),
        "y": list(mydataset_df['Trigger5'][:2]),
        "text":"YP Within Limits but large increase detected – recommend re-testing titrations",
        "mode": "markers",
        "marker": {
            #"sizemode": "area",
            #"sizeref": 200000,
            "size": 10,
            "opacity":0.5
        }
        }
sub_fig_data["data"].append(data_dict)

data_dict = {
        "x": list(mydataset_df['Full Date_parsed'][:2]),
        "y": list(mydataset_df['Trigger6'][:2]),
        "text":"YP Within Limits but large decrease detected – recommend re-testing titrations",
        "mode": "markers",
        "marker": {
            #"sizemode": "area",
            #"sizeref": 200000,
            "size": 10,
            "opacity":0.5
        }
        }
sub_fig_data["data"].append(data_dict)
#list_component = ["Lime"] 

for component in list_component:
    

    data_dict = {
            "x": list(mydataset_df['Full Date_parsed'][:2]),
            "y": list(mydataset_df[component][:2]),
            #"mode": "markers",
            "mode": "lines",
            "fill":'tonexty'
            #"marker": {
                #"sizemode": "area",
                #"sizeref": 200000,
            #    "size": 4
            #}
            }
    sub_fig_data["data"].append(data_dict)




sub_figure = go.Figure(data=sub_fig_data)

number_of_traces = len(sub_fig_data["data"])

l = list(range(number_of_traces))
print(l)
#print(sub_fig_data["data"])



# make frames
#frame = {"data": [], "name":"2020-06-26 04:00:00","traces":[0, 1, 2,3,4,5,6,7] }
frame = {"data": [], "name":"2020-06-26 04:00:00","traces":l }
for k  in  mydataset_df['theDate']:
    #frame = {"data": [], "name": str(k),"traces":[0, 1, 2,3,4,5,6,7]}
    frame = {"data": [], "name": str(k),"traces":l}
    dataset_by_date = mydataset_df[mydataset_df['theDate']<=k]
    
    data_dict = {
        "x": list(dataset_by_date['Full Date_parsed']),
        "y": list(dataset_by_date['YP']),
        "mode": "markers",
        "marker": {
            #"sizemode": "area",
            #"sizeref": 200000,
            "size": 4
        }
        
        }
    frame["data"].append(data_dict)
    
    data_dict = {
        "x": list(dataset_by_date['Full Date_parsed']),
        "y": list(dataset_by_date['Trigger1']),
        "mode": "markers",
        "text":"YP is too low, please add TRUVIS",
        "marker": {
            "sizemode": "area",
            #"sizeref": 200000,
            "size": 10,
            "opacity":0.5
        }
        
        }   
    frame["data"].append(data_dict)
    data_dict = {
        "x": list(dataset_by_date['Full Date_parsed']),
        "y": list(dataset_by_date['Trigger2']),
        "mode": "markers",
        "text":"YP is too high, please add Diesel",
        "marker": {
            "sizemode": "area",
            #"sizeref": 200000,
            "size": 10,
            "opacity":0.5
        }
        
        }   
    frame["data"].append(data_dict)
    
        
    data_dict = {
        "x": list(dataset_by_date['Full Date_parsed']),
        "y": list(dataset_by_date['Trigger3']),
        "mode": "markers",
        "text":"Increasing YP trend – Within limits but consider adding Diesel",
        "marker": {
            "sizemode": "area",
            #"sizeref": 200000,
            "size": 10,
            "opacity":0.5
        }
        
        }   
    frame["data"].append(data_dict)
    
    data_dict = {
        "x": list(dataset_by_date['Full Date_parsed']),
        "y": list(dataset_by_date['Trigger4']),
        "mode": "markers",
        "text":"Decreasing YP trend – Within limits but consider adding TRUVIS",
        "marker": {
            "sizemode": "area",
            #"sizeref": 200000,
            "size": 10,
            "opacity":0.5
        }
        
        }   
    frame["data"].append(data_dict)    
    
    data_dict = {
        "x": list(dataset_by_date['Full Date_parsed']),
        "y": list(dataset_by_date['Trigger5']),
        "mode": "markers",
        "text":"YP Within Limits but large increase detected – recommend re-testing titrations",
        "marker": {
            "sizemode": "area",
            #"sizeref": 200000,
            "size": 10,
            "opacity":0.5
        }
        
        }   
    frame["data"].append(data_dict)  
    
    
    data_dict = {
        "x": list(dataset_by_date['Full Date_parsed']),
        "y": list(dataset_by_date['Trigger6']),
        "mode": "markers",
        "text":"YP Within Limits but large decrease detected – recommend re-testing titrations",
        "marker": {
            "sizemode": "area",
            #"sizeref": 200000,
            "size": 10,
            "opacity":0.5
        }
        
        }   
    frame["data"].append(data_dict)

    for component in list_component:
    

        data_dict = {
                "x": list(dataset_by_date['Full Date_parsed']),
                "y": list(dataset_by_date[component]),
                "mode": "lines",
                "fill":'tonexty'
                #"marker": {
                    #"sizemode": "area",
                    #"sizeref": 200000,
                #    "size": 4
                #}
            }
    
 
        frame["data"].append(data_dict)

    fig_dict["frames"].append(frame)
    #print(frame)
    #print('####################################################################################################')
    slider_step = {"args": [
        [k],
        {"frame": {"duration": 300, "redraw": False},
         "mode": "immediate",
         "transition": {"duration": 0}}
    ],
        "label": k,
        "method": "animate"}
    sliders_dict["steps"].append(slider_step)


fig_dict["layout"]["sliders"] = [sliders_dict]




[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]


In [0]:
fig = make_subplots(rows=2, cols=1,shared_xaxes=True)
sub_row = 1
for t in sub_figure.data:
    if sub_row >= 8:
        fig.add_trace(t, row=2, col=1)
        
    else:
        
        fig.add_trace(t, row=1, col=1)
    sub_row = sub_row +1
    
    
    
    #if sub_row == 3:
     #     break
fig.update_layout(yaxis2 = dict(range=[treatment_min_y, treatment_max_y]))
fig.update_layout(yaxis1 = dict(range=[min_y, max_y]))

fig.update_layout(xaxis2 = dict(range=[min_x, max_x]))
fig.update_layout(xaxis1 = dict(range=[min_x, max_x]))
#fig.update_xaxes("range"= [min_x, max_x])
fig.update_layout(width=1010, height=950)

fig.update(frames=fig_dict["frames"])
fig.update(layout=fig_dict["layout"]),
#fig.update_layout(updatemenus=fig_dict["layout"]["updatemenus"],
#                  sliders=fig_dict["layout"]["sliders"])


fig.add_shape(type='line',
                x0=min_x,
                y0=22,
                x1=max_x,
                y1=22,
                line=dict(color='Red',),
                xref='x',
                yref='y'
)

fig.add_shape(type='line',
                x0=min_x,
                y0=14,
                x1=max_x,
                y1=14,
                line=dict(color='Red',),
                xref='x',
                yref='y'
)


In [0]:
fig.show()

In [0]:
print("done")

In [1]:
import chart_studio.plotly as py
import plotly.graph_objects as go

import numpy as np

s = np.linspace(0, 2 * np.pi, 240)
t = np.linspace(0, np.pi, 240)
tGrid, sGrid = np.meshgrid(s, t)

r = 2 + np.sin(7 * sGrid + 5 * tGrid)  # r = 2 + sin(7s+5t)
x = r * np.cos(sGrid) * np.sin(tGrid)  # x = r*cos(s)*sin(t)
y = r * np.sin(sGrid) * np.sin(tGrid)  # y = r*sin(s)*sin(t)
z = r * np.cos(tGrid)                  # z = r*cos(t)

surface = go.Surface(x=x, y=y, z=z)
data = [surface]

layout = go.Layout(
    title='Parametric Plot',
    scene=dict(
        xaxis=dict(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=True,
            backgroundcolor='rgb(230, 230,230)'
        ),
        yaxis=dict(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=True,
            backgroundcolor='rgb(230, 230,230)'
        ),
        zaxis=dict(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=True,
            backgroundcolor='rgb(230, 230,230)'
        )
    )
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='jupyter-parametric_plot')

ModuleNotFoundError: No module named 'chart_studio'